<a href="https://colab.research.google.com/github/Nargiz-chess/Solar_forecasting/blob/main/LSTM_Germany.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

from pandas import read_csv
import pandas as pd
import datetime as dt
import matplotlib
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

# multivariate multi-step encoder-decoder lstm
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

Mounted at /content/drive


In [ ]:
path1 = '/content/drive/MyDrive/Master_thesis/Datasets/Gerdata_clean.csv'
path2 = '/content/drive/MyDrive/Master_thesis/Datasets/irradiance.csv'

In [ ]:
df1 = read_csv(path1, header =0, index_col=0) 

df.head()
df.columns

Index(['DE_KN_industrial1_pv_1', 'DE_KN_industrial1_pv_2',
       'DE_KN_industrial2_pv', 'DE_KN_industrial3_pv_facade',
       'DE_KN_industrial3_pv_roof', 'DE_KN_residential1_pv',
       'DE_KN_residential3_pv', 'DE_KN_residential4_pv',
       'DE_KN_residential6_pv'],
      dtype='object')

In [ ]:
df.shape

(6588, 9)

In [ ]:
array = df.values

Gerdata = array.reshape(array.shape[0]*array.shape[1], order="F")
print(Gerdata.shape)

(59292,)
[0. 0. 0. ... 0. 0. 0.]


In [ ]:
df2=read_csv(path2, header =0, index_col=0) 
df2.head()

df2=df2.drop("local_time", axis=1)
df2.head()

,time,radiation_surface
local_time,,
2016-02-01 05:00,2016-02-01 04:00:00,0.000
2016-02-01 06:00,2016-02-01 05:00:00,0.000
2016-02-01 07:00,2016-02-01 06:00:00,0.467
2016-02-01 08:00,2016-02-01 07:00:00,31.309
2016-02-01 09:00,2016-02-01 08:00:00,112.503


In [ ]:
array2 = df2.values

print(array2)
print(array2.shape)

weath_data = array2.reshape(array2.shape[0]*array2.shape[1])

print(weath_data)

[[0.   ]
 [0.   ]
 [0.467]
 ...
 [0.   ]
 [0.   ]
 [0.   ]]
(6588, 1)
[0.    0.    0.467 ... 0.    0.    0.   ]


In [ ]:
len(df.columns)

#stacking irradiance 9 times to match with each house in the datset
weath_data = np.tile(weath_data,9)



In [ ]:
weath_data.shape #validate" same length as the Gerdata

(59292,)

In [ ]:
data= np.column_stack((Gerdata,weath_data))
data.shape

(59292, 2)

In [ ]:
# with open ("/content/drive/MyDrive/Master_thesis/Datasets/Data_to_supervised.npy", "wb") as a:
#    np.save(a,data_csv)

# array = np.load("/content/drive/MyDrive/Master_thesis/Datasets/Data_to_supervised.npy")

#Save the data that is in the supervised time series format
np.savetxt("/content/drive/MyDrive/Master_thesis/Datasets/Data_to_supervised.npy", data) 



In [ ]:
data=np.loadtxt("/content/drive/MyDrive/Master_thesis/Datasets/Data_to_supervised.npy")
np.shape(data)

In [ ]:
# split the dataset into train/test sets & scale

def split_data(data=data):
  # split 
  train, test = data[:-18], data[-18:]   
  return train,test
   
def scaled(scaler_name=MinMaxScaler):
  scaler = scaler_name()
  train,test = split_data()
  scaled_train =  scaler.fit_transform(train)
  scaled_test= scaler.transform(test)
  return scaled_train, scaled_test, scaler

# restructure into windows	
def restructured_3D_data():
  train,test,_ =  scaled()
  train = array(split(train, len(train)/18))
  test = array(split(test, len(test)/18))
  return train, test


# evaluate  forecasts against expected values
def evaluate_forecasts(actual, predicted):
	scores = list()
	# calculate an RMSE score for each day
	for i in range(actual.shape[1]):
		# calculate mse
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# calculate rmse
		rmse = sqrt(mse)
		# store
		scores.append(rmse)
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores
 
# summarize scores
def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('%s: [%.3f] %s' % (name, score, s_scores))
 
# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=18):
	# flatten data
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	# step over the entire history one time step at a time
	for _ in range(len(data)):
		# define the end of the input sequence
		in_end = in_start + n_input
		out_end = in_end + n_out
		# ensure we have enough data for this instance
		if out_end <= len(data):
			X.append(data[in_start:in_end, :])
			y.append(data[in_end:out_end, 0])
		# move along one time step
		in_start += 1
	return array(X), array(y)
 
# train the model
def build_model(train, n_input):
	# prepare data
	train_x, train_y = to_supervised(train, n_input)
	# define parameters
	verbose, epochs, batch_size = 1, 20, 17
	n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
	# reshape output into [samples, timesteps, features]
	train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
	# define model
	model = Sequential()
	model.add(LSTM(100, activation='relu', input_shape=(n_timesteps, n_features)))
	model.add(RepeatVector(n_outputs))
	model.add(LSTM(100, activation='relu', return_sequences=True))
	model.add(TimeDistributed(Dense(50, activation='relu')))
	model.add(TimeDistributed(Dense(1)))
	model.compile(loss='mse', optimizer='adam')
	# fit network
	model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
	return model
 
# make a forecast
def forecast(model, history, n_input):
	# flatten data
	data = array(history)
	data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
	# retrieve last observations for input data
	input_x = data[-n_input:, :]
	# reshape into [1, n_input, n]
	input_x = input_x.reshape((1, input_x.shape[0], input_x.shape[1]))
	# forecast the next time steps
	yhat = model.predict(input_x, verbose=0)
	# we only want the vector forecast
	yhat = yhat[0]
	return yhat
 
# evaluate a single model
def evaluate_model(train, test, n_input):
	# fit model
  model = build_model(train, n_input)
	# history is a list of weekly data
  history = [x for x in train]
	# walk-forward validation over each week
  predictions = list()
  for i in range(len(test)):
		# predict the week
    yhat_sequence = forecast(model, history, n_input)
		# store the predictions
    predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next week
    history.append(test[i, :])
	# evaluate predictions days for each week
  predictions = array(predictions)
  score, scores = evaluate_forecasts(test[:, :, 0], predictions)
  return score, scores, predictions

def inverse_predictions(prediction_array, scaler_name = MinMaxScaler):
  train, test = split_data()
  scaler= scaler_name()
  PV_data = np.reshape(test[:,0], (15,1))
  predictions = np.reshape(prediction_array,(15,1))
  test_PV_col = scaler.fit_transform(PV_data)
  prediction_array = scaler.inverse_transform(predictions)
  return prediction_array

# split into train and test & scale
train, test  = restructured_3D_data()
# evaluate model and get scores
n_input =  18
score, scores,scaled_predictions = evaluate_model(train, test, n_input)

# summarize scores
summarize_scores('lstm', score, scores)
print(summarize_scores)

# predictions = inverse_predictions(scaled_predictions)
# print(predictions)


Epoch 1/20
3485/3485 [==============================] - 81s 23ms/step - loss: nan
Epoch 2/20
 233/3485 [=>............................] - ETA: 1:11 - loss: nan

KeyboardInterrupt: ignored